In [29]:
import pandas as pd
df = pd.read_csv('ITO_2years.csv')

In [54]:
import ollama

# Pull the 'llama2' model
ollama.pull('mistral')
ollama.pull('llama2')
ollama.pull('orca-mini')
# Rest of the code
#response = ollama.chat(model='llama2', messages=[
#    {
#        'role': 'user',
#        'content': 'Why is the sky blue?',
#    },
#])
#print(response['message']['content'])

{'status': 'success'}

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55499 entries, 0 to 55498
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   station      55499 non-null  object 
 1   city         55499 non-null  object 
 2   dd-mmm-yyyy  55499 non-null  object 
 3   hh:mm        55499 non-null  object 
 4   year         55499 non-null  int64  
 5   month        55499 non-null  int64  
 6   day          55499 non-null  int64  
 7   hour         55499 non-null  int64  
 8   value        55499 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 3.8+ MB


In [32]:
df_ = df[:10000]
df_['datetime'] = pd.to_datetime(df_['dd-mmm-yyyy'] + ' ' + df_['hh:mm'], format='%d-%b-%Y %H:%M')

# Last datetime in your data
last_datetime = df_['datetime'].max()

C:\Users\Aditya\AppData\Local\Temp\ipykernel_31244\2923763294.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['datetime'] = pd.to_datetime(df_['dd-mmm-yyyy'] + ' ' + df_['hh:mm'], format='%d-%b-%Y %H:%M')


In [33]:
df_['datetime'] = pd.to_datetime(df_['dd-mmm-yyyy'] + ' ' + df_['hh:mm'], format='%d-%b-%Y %H:%M')

# Last datetime in your data
last_datetime = df_['datetime'].max()

C:\Users\Aditya\AppData\Local\Temp\ipykernel_31244\4162966839.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_['datetime'] = pd.to_datetime(df_['dd-mmm-yyyy'] + ' ' + df_['hh:mm'], format='%d-%b-%Y %H:%M')


In [34]:
last_month_pm25 = df_['value'].tail(25 * 24 * 4).tolist()
#predicted_value = forecast_value_with_llama2_chat(last_month_pm25)
#predicted_value

In [35]:
#print(last_month_pm25)

In [46]:
# Remove decimal parts
pm25_values = [int(value) if value.is_integer() else value for value in last_month_pm25]

# Tokenization: Tokenize each numerical value into individual digits
def tokenize_number(number):
    return ''.join(str(digit) for digit in str(number))

# Formatting: Format the numerical values in a consistent manner
formatted_pm25_values = ', '.join(tokenize_number(value) for value in pm25_values)

# Encoding: Encode the formatted numerical values into a text format
encoded_text = formatted_pm25_values

# Now 'encoded_text' contains the PM2.5 values without decimal parts, suitable for LLM input
print("Encoded Text:", encoded_text)

Encoded Text: 51, 47, 44, 43, 42, 43, 41, 39, 40, 44, 48, 53, 50, 42, 39, 34, 35, 35, 34, 32, 33, 37, 39, 40, 41, 43, 46, 50, 53, 57, 59, 60, 62, 65, 66, 66, 65, 67, 72, 76, 75, 68, 61, 57, 57, 57, 60, 65, 66, 67, 65, 60, 56, 53, 49, 46, 43, 42, 42, 41, 42, 41, 40, 40, 39, 41, 44, 50, 59, 73, 92, 110, 119, 118, 106, 93, 83, 79, 80, 86, 90, 90, 86, 81, 80, 85, 93, 100, 105, 105, 96, 85, 75, 67, 64, 60, 54, 48, 44, 41, 40, 39, 37, 36, 34, 33, 32, 31, 32, 34, 35, 37, 39, 37, 37, 37, 37, 37, 37, 38, 39, 39, 40, 41, 45, 50, 53, 56, 58, 58, 60, 61, 64, 68, 71, 75, 76, 74, 74, 71, 65, 61, 57, 55, 54, 54, 54, 55, 55, 54, 53, 50, 52, 52, 53, 54, 57, 65, 78, 95, 108, 115, 115, 107, 99, 93, 93, 99, 107, 116, 123, 126, 126, 126, 127, 127, 123, 116, 108, 100, 96, 93, 90, 88, 89, 93, 101, 111, 114, 106, 93, 78, 66, 57, 54, 3, 12, 20, 28, 33, 33, 34, 35, 35, 34, 33, 33, 33, 35, 37, 39, 40, 41, 41, 42, 45, 48, 54, 60, 65, 69, 71, 73, 76, 80, 89, 96, 101, 103, 101, 98, 95, 90, 85, 82, 81, 83, 85, 86, 8

In [47]:
# Function to generate a well-structured prompt for time series forecasting
def generate_time_series_forecasting_prompt(historical_data, prediction_horizon):
    # Generate the prompt
    prompt = f"Given the following time series data {historical_data}. Predict the next {predicted_horizon} values in the sequence."
    return prompt

predicted_horizon = 96
prompt = generate_time_series_forecasting_prompt(encoded_text, predicted_horizon)
print("Generated Prompt:", prompt)
prompt2 = 'What would be the air quality forecast for ITO station in delhi for 26 april 2017'

Generated Prompt: Given the following time series data 51, 47, 44, 43, 42, 43, 41, 39, 40, 44, 48, 53, 50, 42, 39, 34, 35, 35, 34, 32, 33, 37, 39, 40, 41, 43, 46, 50, 53, 57, 59, 60, 62, 65, 66, 66, 65, 67, 72, 76, 75, 68, 61, 57, 57, 57, 60, 65, 66, 67, 65, 60, 56, 53, 49, 46, 43, 42, 42, 41, 42, 41, 40, 40, 39, 41, 44, 50, 59, 73, 92, 110, 119, 118, 106, 93, 83, 79, 80, 86, 90, 90, 86, 81, 80, 85, 93, 100, 105, 105, 96, 85, 75, 67, 64, 60, 54, 48, 44, 41, 40, 39, 37, 36, 34, 33, 32, 31, 32, 34, 35, 37, 39, 37, 37, 37, 37, 37, 37, 38, 39, 39, 40, 41, 45, 50, 53, 56, 58, 58, 60, 61, 64, 68, 71, 75, 76, 74, 74, 71, 65, 61, 57, 55, 54, 54, 54, 55, 55, 54, 53, 50, 52, 52, 53, 54, 57, 65, 78, 95, 108, 115, 115, 107, 99, 93, 93, 99, 107, 116, 123, 126, 126, 126, 127, 127, 123, 116, 108, 100, 96, 93, 90, 88, 89, 93, 101, 111, 114, 106, 93, 78, 66, 57, 54, 3, 12, 20, 28, 33, 33, 34, 35, 35, 34, 33, 33, 33, 35, 37, 39, 40, 41, 41, 42, 45, 48, 54, 60, 65, 69, 71, 73, 76, 80, 89, 96, 101, 103, 1

LLAMA2

In [48]:
def forecast_with_llama_chat(historical_data, prediction_horizon):
    # Generate the prompt
    prompt = generate_time_series_forecasting_prompt(historical_data, prediction_horizon)

    # Get Llama 2 response as user message
    response = ollama.chat(model='llama2', messages=[
        {
            'role': 'user',
            'content': prompt.format(historical_data=historical_data, prediction_horizon=prediction_horizon)
        }
    ])

    predicted_value = None
    try:
        predicted_value = response['message']['content']
    except:
        print("Error extracting value from response. Skipping this prediction.")

    return predicted_value

In [57]:
predicted_value = forecast_with_llama_chat(encoded_text, predicted_horizon)
predicted_value

'\nTo predict the next 96 values in the sequence, we can use the pattern and structure of the sequence to make educated guesses about the next values. Here are our predictions:\n\n0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96.\n\nOf course, these are just our predictions based on the pattern and structure of the sequence so far. There is no guarantee that the next values will actually follow this pattern or be exactly these numbers.'

MISTRAL

In [50]:
def forecast_with_mistral_chat(historical_data, prediction_horizon):
    # Generate the prompt
    prompt = generate_time_series_forecasting_prompt(historical_data, prediction_horizon)

    # Get Llama 2 response as user message
    response = ollama.chat(model='mistral', messages=[
        {
            'role': 'user',
            'content': prompt.format(historical_data=historical_data, prediction_horizon=prediction_horizon)
        }
    ])

    predicted_value_ = None
    try:
        predicted_value_ = response['message']['content']
    except:
        print("Error extracting value from response. Skipping this prediction.")

    return predicted_value_

In [51]:
predicted_value_ = forecast_with_mistral_chat(encoded_text, predicted_horizon)
predicted_value_

" The given sequence is a combination of two sequences: A075116 and A115328. The first sequence (A075116) generates the odd indices, while the second sequence (A115328) generates the even indices.\n\nLet's find the next 48 values for each sequence separately:\n\nSequence A075116:\nThe formula for this sequence is given by oE4(n), where n is a non-negative integer. Using this information, we can calculate the next 48 terms as follows:\n\n1, 3, 5, 15, 21, 45, 63, 105, 143, 221, 309, 420, 579, 792, 1089, 1547, 2205, 3221, 4624, 6539, 9841, 13845, 19991, 29762, 46305, 69811, 109464, 170796, 259697, 423303, 662037, 1031405, 1640231, 2580067, 4202935, 6696765, 10610221, 16841765, 27317781, 45364519, ...\n\nNext terms: 75116th term = oE4(48) = 2759732521\n                           => Next terms: 2759732521, 2759732523, ...\n\nSequence A115328:\nThe formula for this sequence is given by e(n^2), where n is a non-negative integer. Using this information, we can calculate the next 48 terms as fo

ORCA-MINI

In [55]:
def forecast_with_gemma_chat(historical_data, prediction_horizon):
    # Generate the prompt
    prompt = generate_time_series_forecasting_prompt(historical_data, prediction_horizon)

    # Get Llama 2 response as user message
    response = ollama.chat(model='orca-mini', messages=[
        {
            'role': 'user',
            'content': prompt.format(historical_data=historical_data, prediction_horizon=prediction_horizon)
        }
    ])

    predicted_value__ = None
    try:
        predicted_value__ = response['message']['content']
    except:
        print("Error extracting value from response. Skipping this prediction.")

    return predicted_value__

In [56]:
predicted_value__ = forecast_with_gemma_chat(encoded_text, predicted_horizon)
predicted_value__

' The sequence is generated by multiplying the previous number by 2 and then adding 2 to it. So, the next 96 values in the sequence would be:\n\n73, 81, 89, 97, 104, 111, 118, 125, 132, 139, 146, 153, 160, 167, 174, 181, 188, 195, 202, 210, 218, 226, 233, 241, 249, 256, 263, 270, 277, 284, 291, 300, 308, 316, 324, 332, 340, 348, 356, 364, 372, 380, 388, 396, 404, 412, 420, 428, 436, 444, 452, 460, 468, 476, 484, 492, 500, 508, 516, 524, 532, 540, 548, 556, 564, 572, 580, 588, 596, 604, 612, 620, 628, 636, 644, 652, 660, 668, 676, 684, 692, 700, 708, 716, 724, 732, 740, 748, 756, 764, 772, 780, 788, 796, 804, 812, 820, 828, 836, 844, 852, 860, 868, 876, 884, 892, 900, 908, 916, 924, 932, 940, 948, 956, 964, 972, 980, 988, 996, 1004, 1012, 1020, 1028, 1036, 1044, 1052, 1060, 1068, 1076, 1084, 1092, 1100, 1108, 1116, 1124, 1132, 1140, 1148, 1156, 1164, 1172, 1180, 1188, 1196, 1204, 1212, 1220, 1228, 1236, 1244, 1252, 1260, 1268, 1276, 1284, 1292, 1300, 1308, 1316, 1324, 1332, 134'

Even after using the same prompt from the paper 'LLM are Zero-Shot Time Series Forecasters' the results from all the models used were clearly not good.